In [2]:
import os
import nltk
import nltk.corpus
from nltk.corpus import stopwords
from nltk.stem import wordnet,WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import keras
import tensorflow
from keras.layers import Embedding, LSTM, Bidirectional, Dense,Dropout
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix

In [4]:
trainds=pd.read_csv('/content/drive/MyDrive/train.csv')

In [5]:
trainds.dropna(axis=0,inplace=True)


In [6]:
trainds.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [39]:
trainds.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
X=trainds.drop('label',axis=1)
y=trainds['label']

In [8]:
messages=X.copy()

In [9]:
messages.drop('id',inplace=True,axis=1)

In [10]:
messages.reset_index(inplace=True)

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
vocab_size=5000

In [15]:
onehot_repr=[one_hot(words,vocab_size)for words in corpus]

In [16]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2339 3560  600]
 [   0    0    0 ... 3555 3601 2037]
 [   0    0    0 ...  153 1547 1864]
 ...
 [   0    0    0 ... 3056 4562  750]
 [   0    0    0 ... 2202 4082 4910]
 [   0    0    0 ...  910 3488  327]]


In [45]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [46]:
len(embedded_docs),y.shape

(18285, (18285,))

In [19]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [20]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [48]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 17s 72ms/step - loss: 0.3498 - accuracy: 0.8318 - val_loss: 0.1929 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1456 - accuracy: 0.9439 - val_loss: 0.1989 - val_accuracy: 0.9208
Epoch 3/10
192/192 [==============================] - 2s 11ms/step - loss: 0.1079 - accuracy: 0.9608 - val_loss: 0.2030 - val_accuracy: 0.9198
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0810 - accuracy: 0.9727 - val_loss: 0.2262 - val_accuracy: 0.9208
Epoch 5/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0600 - accuracy: 0.9807 - val_loss: 0.2652 - val_accuracy: 0.9132
Epoch 6/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0465 - accuracy: 0.9849 - val_loss: 0.3186 - val_accuracy: 0.9122
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0337 - accuracy: 0.9879 - val_loss: 0.4000 - val_accuracy: 0.913

In [23]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0)

189/189 [==============================] - 1s 2ms/step


In [24]:
confusion_matrix(y_test,y_pred)

array([[3053,  366],
       [ 166, 2450]])

In [50]:
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 20s 72ms/step - loss: 0.2945 - accuracy: 0.8654 - val_loss: 0.1990 - val_accuracy: 0.9193
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1440 - accuracy: 0.9459 - val_loss: 0.1978 - val_accuracy: 0.9196
Epoch 3/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1053 - accuracy: 0.9634 - val_loss: 0.2217 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0829 - accuracy: 0.9732 - val_loss: 0.2652 - val_accuracy: 0.9135
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0632 - accuracy: 0.9778 - val_loss: 0.3347 - val_accuracy: 0.9105
Epoch 6/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0533 - accuracy: 0.9829 - val_loss: 0.3054 - val_accuracy: 0.9075
Epoch 7/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0369 - accuracy: 0.9874 - val_loss: 0.3539 - val_accuracy: 0.907

In [26]:
from tensorflow.keras.models import save_model
model.save('/content/drive/MyDrive/Models/newsmodel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [63]:
input_sentence = "Rakesh Jhunjhunwala, Dolly Khanna, Ashish Kacholia increased stake in these firms in Q3"
revie = re.sub('[^a-zA-Z]', ' ', input_sentence)
revie = revie.lower()
revie = revie.split()
revie = [ps.stem(word) for word in revie if not word in stopwords.words('english')]
revie = ' '.join(revie)
print(revie)
input_sentence_onehot = [one_hot(revie, vocab_size)]
input_sentence_padded = pad_sequences(input_sentence_onehot)
prediction = model.predict(input_sentence_padded)
print(prediction)

rakesh jhunjhunwala dolli khanna ashish kacholia increas stake firm q
1/1 [==============================] - 0s 70ms/step
[[0.00257643]]
